## **임대주택 단지 내 적정 주차수요 예측**

#### 주제 
유형별 임대주택 설계 시 단지 내 적정 주차 수요 예측


#### **평가**  
- MAE(Mean Absolute Error)
- public : 전체 TEST 데이터 중 무작위 33%
- private : 전체 TEST 데이터 중 무작위 67%

#### 외부 데이터 및 사전 학습 모델
- 공공 데이터와 같이 누구나 얻을 수 있고 법적 제약이 없는 외부 데이터 허용
- 사전학습 모델의 경우 사전학습에 사용된 데이터를 명시
- 최종 평가시 외부데이터 및 출처 제출

#### 데이터 오류 처리 방안
 
-  테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지  
   ['C2675','C2335', 'C1327']
-  Train 제거  
   ['C2085', 'C1397', 'C2431', 'C1649', 'C1036', 'C2675', 'C2335', 'C1327', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']






In [106]:
import pandas
import numpy
import os
import math
import warnings
from IPython.core.display import display, HTML

## 전체 구간을 넓게
display(HTML("<style>.container { width:100% !important; }</style>"))
## 각 컬럼 width 최대로
pandas.set_option('display.max_colwidth', -1)
## rows 500
pandas.set_option('display.max_rows', 500)
## columns
pandas.set_option('display.max_columns', 500)
pandas.set_option('display.width', 1000)

pandas.options.display.float_format = '{:.5f}'.format
warnings.filterwarnings(action = 'ignore')

PATH = 'C:/Users/WAI/OneDrive/04.DOMHWANGCHA/DOMHWANGCHA/99.DATA/98.DACON/01.주차수요예측AI경진대회/'

In [3]:
train_df = pandas.read_csv(PATH + "train.csv", index_col = False)
test_df = pandas.read_csv(PATH + "test.csv", index_col = False)
df = train_df.append(test_df)
age_df = pandas.read_csv(PATH + "age_gender_info.csv", index_col=False)

print("Training Set Shape : {}".format(train_df.shape))
print("Test Set Shape : {}".format(test_df.shape))
print("Total Set Shape : {}".format(df.shape))
print("Age Gender Set Shape : {}".format(age_df.shape))

Training Set Shape : (2952, 15)
Test Set Shape : (1022, 14)
Total Set Shape : (3974, 15)
Age Gender Set Shape : (16, 23)


In [19]:
# sweetviz 라이브러리를 사용하여 EDA 결과 html로 출력
import sweetviz

eda_report = sweetviz.compare(train_df, test_df, "등록차량수")
eda_report.show_html()


Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)


Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [42]:

train_df.loc[train_df["단지코드"] == "C1350"]["전용면적별세대수"].sum()
train_df.loc[train_df["단지코드"] == "C1350"]
# 분양의 경우 임대료 X
# 

print("전체 아파트 단지 : {}".format(len(train_df["단지코드"].unique())))
print("공급유형 : {}\n".format(train_df["공급유형"].value_counts()))
print("임대건물구분: {}\n".format(train_df["임대건물구분"].value_counts()))
#train_df.groupby(["단지코드", "임대건물구분"]).count() # Pivot으로 아파트 ,상가 단지 구분

전체 아파트 단지 : 423
공급유형 : 국민임대         1758
임대상가         562 
행복주택         213 
공공임대(10년)    205 
영구임대         152 
공공임대(50년)    31  
공공임대(분납)     12  
장기전세         9   
공공분양         7   
공공임대(5년)     3   
Name: 공급유형, dtype: int64

임대건물구분: 아파트    2390
상가     562 
Name: 임대건물구분, dtype: int64



In [77]:
# Train Set EDA
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
총세대수,2952.0,8.866612e+02,5.135402e+02,2.600000e+01,5.135000e+02,779.00,1.106000e+03,2.568000e+03
전용면적,2952.0,4.475722e+01,3.187428e+01,1.262000e+01,3.210000e+01,39.93,5.156250e+01,5.834000e+02
전용면적별세대수,2952.0,1.027480e+02,1.326402e+02,1.000000e+00,1.400000e+01,60.00,1.440000e+02,1.865000e+03
공가수,2952.0,1.292107e+01,1.077883e+01,0.000000e+00,4.000000e+00,11.00,2.000000e+01,5.500000e+01
임대보증금,2371.0,2.686432e+07,2.161391e+07,3.056000e+06,1.441800e+07,20971000.00,3.245900e+07,2.164230e+08
임대료,2362.0,1.957181e+05,1.303745e+05,1.665000e+04,1.108500e+05,161270.00,2.365375e+05,1.058030e+06
도보 10분거리 내 지하철역 수(환승노선 수 반영),2741.0,1.765779e-01,4.274079e-01,0.000000e+00,0.000000e+00,0.00,0.000000e+00,3.000000e+00
도보 10분거리 내 버스정류장 수,2948.0,3.695726e+00,2.644665e+00,0.000000e+00,2.000000e+00,3.00,4.000000e+00,2.000000e+01
단지내주차면수,2952.0,6.016684e+02,3.964071e+02,1.300000e+01,2.792500e+02,517.00,8.230000e+02,1.798000e+03
등록차량수,2952.0,5.597683e+02,4.333750e+02,1.300000e+01,2.200000e+02,487.00,7.700000e+02,2.550000e+03


In [78]:
# Null 값 확인
# 임대보증금, 임대료 Null 존재 
train_df.isnull().sum().to_frame().T

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,평수,공급평수,전용면적평수
0,0,0,0,0,0,0,0,0,0,581,590,211,4,0,0,0,0,0


In [125]:
# 임대보증금과 임대료가 Null 또는 비어있는 값으로 들어가 있는 경우 0 처리
# "-" 문자열 Null로 치환 

train_df = train_df.replace("-", numpy.nan)
train_df = train_df.astype({"임대보증금" : float, "임대료" : float}) 

# 1. 임대보증금, 임대료가 NaN인 데이터에서 상가 아파트에 따른 정보 확인 
print("임대보증금이 임대건물구분별 Null 데이터 : \n {}".format(train_df[train_df["임대보증금"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
print("임대보증금이 자격유형별 Null 데이터 : \n {}".format(train_df[train_df["임대보증금"].isnull() == True].groupby(["자격유형"]).count()["단지코드"]))
print("---------------------------------------------------------")
# 상가 : 562건, 아파트: 19 건
train_df_apt_null = train_df[train_df["임대보증금"].isnull() == True].loc[train_df["임대건물구분"] == "아파트"]
print("결측이 존재하는 아파트 건수 : {}".format(len(train_df_apt_null)))
train_df_store_null = train_df[train_df["임대보증금"].isnull() == True].loc[train_df["임대건물구분"] == "상가"]

# 아파트의 임대 보증금 Null데이터 관측치 확인
print("Null이 존재하는 Apt의 단지코드 : {}".format(train_df_apt_null["단지코드"].unique()))
# Null 이존재하는 단지코드
# ['C1350', 'C1326', 'C1786', 'C2186']
# 단지코드만 조회
train_df_apt_null.loc[train_df["단지코드"].isin(['C1350', 'C1326', 'C1786', 'C2186'])]
# 대전, 부산, 강원도, 대구 지역에 있는 아파트의 임대보즘금의 Null 값이 발생
# 전용면적별, 공급유형별 임대보증금을 비교하여 Null 값 보정 가능 할 수 도 있음


임대보증금이 임대건물구분별 Null 데이터 : 
 임대건물구분
상가     562
아파트    19 
Name: 단지코드, dtype: int64
임대보증금이 자격유형별 Null 데이터 : 
 자격유형
D    569
H    8  
K    4  
Name: 단지코드, dtype: int64
---------------------------------------------------------
결측이 존재하는 아파트 건수 : 19
Null이 존재하는 Apt의 단지코드 : ['C1350' 'C1326' 'C1786' 'C2186']


,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,평수,공급평수,전용면적평수,평수범주
2331,C1350,1401,아파트,대전광역시,공공분양,74.94000,317,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,22.70909,22.80000,22.80000,6001
2332,C1350,1401,아파트,대전광역시,공공분양,74.94000,137,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,22.70909,22.80000,22.80000,6001
2333,C1350,1401,아파트,대전광역시,공공분양,74.94000,22,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,22.70909,22.80000,22.80000,6001
2334,C1350,1401,아파트,대전광역시,공공분양,84.94000,164,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,25.73939,25.80000,25.80000,7001
2335,C1350,1401,아파트,대전광역시,공공분양,84.94000,19,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,25.73939,25.80000,25.80000,7001
2336,C1350,1401,아파트,대전광역시,공공분양,84.96000,26,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,25.74545,25.80000,25.80000,7001
2337,C1350,1401,아파트,대전광역시,공공분양,84.97000,26,2.00000,D,NaN,NaN,NaN,6.00000,1636.00000,2315.00000,25.74848,25.80000,25.80000,7001
2547,C1326,1934,아파트,부산광역시,국민임대,24.72000,472,43.00000,H,NaN,NaN,0.00000,4.00000,1670.00000,1153.00000,7.49091,7.50000,7.50000,1001
2548,C1326,1934,아파트,부산광역시,국민임대,24.79000,104,43.00000,H,NaN,NaN,0.00000,4.00000,1670.00000,1153.00000,7.51212,7.60000,7.60000,1001
2549,C1326,1934,아파트,부산광역시,국민임대,26.83000,590,43.00000,H,NaN,NaN,0.00000,4.00000,1670.00000,1153.00000,8.13030,8.20000,8.20000,1001


In [127]:

# 전용 면적의 그룹핑필요
# 전용 면적 평수 변환하여 평수별 그룹핑

train_df["전용면적평수"] = [math.ceil(x)/10 for x in train_df["전용면적"]/3.3*10]
# train_df["전용면적평수"].hist()
# 임대 상가의 경우 면적평수가 매우 넓음!
# train_df[train_df["전용면적평수"] > 60]

# 전용평수 범주화
# 아파트 -> 25.8평이 최대 
print("아파트 최대 평수 : {}, 최소 평수 : {}, 평균평수 : {} ".format(train_df_apt["전용면적평수"].max(),train_df_apt["전용면적평수"].min(),round(train_df_apt["전용면적평수"].mean(),1)))
# 범주 구분 
# 원룸 -> 10평 미만 -> 1001
# 10평이상 13평 미만 -> 2001
# 13평이상 18평 미만 -> 3001
# 18평이상 21평 미만 -> 4001
# 21평 이상 24평 미만 -> 5001
# 24평 이상 -> 6001

train_df.loc[train_df["전용면적평수"] < 10, "평수범주"] = 1001
train_df.loc[(train_df["전용면적평수"] >= 10) & (train_df["전용면적평수"] < 13), "평수범주"] = 2001
train_df.loc[(train_df["전용면적평수"] >= 13) & (train_df["전용면적평수"] < 15), "평수범주"] = 3001
train_df.loc[(train_df["전용면적평수"] >= 15) & (train_df["전용면적평수"] < 18), "평수범주"] = 4001
train_df.loc[(train_df["전용면적평수"] >= 18) & (train_df["전용면적평수"] < 21), "평수범주"] = 5001
train_df.loc[(train_df["전용면적평수"] >= 21) & (train_df["전용면적평수"] < 24), "평수범주"] = 6001
train_df.loc[(train_df["전용면적평수"] >= 24) & (train_df["전용면적평수"] < 28), "평수범주"] = 7001
train_df.loc[(train_df["전용면적평수"] >= 28), "평수범주"] = 8001


train_df = train_df.astype({"평수범주" : numpy.int16})



아파트 최대 평수 : 25.8, 최소 평수 : 4.9, 평균평수 : 13.6 


In [123]:

# Null 값을 보정하려면, 상가와 아파트를 구분
# Null이 아닌 데이터 평균 
train_df_apt = train_df[train_df["임대보증금"].isnull() != True].loc[train_df["임대건물구분"] == "아파트"]
train_df_store = train_df[train_df["임대보증금"].isnull() != True].loc[train_df["임대건물구분"] == "상가"]


train_df_area = train_df_apt.loc[train_df_apt["지역"].isin(["대전광역시","부산광역시","강원도","대구광역시"])][["지역","공급유형","평수범주","임대보증금","임대료"]]
train_df_grouped_area = train_df_area.groupby(["지역","공급유형","평수범주"]).mean()

train_df_grouped_area

# 임대상가 여부도 컬럼에 추가 파생변수 생성


임대보증금          임대료
지역    공급유형      평수범주                            
강원도   공공임대(10년) 6001 36494000.00000 514000.00000
                7001 46260000.00000 580820.00000
      공공임대(50년) 2001 9878500.00000  167590.00000
                3001 10244000.00000 152280.00000
                4001 12669666.66667 204430.00000
      국민임대      1001 8880818.18182  115910.90909
                2001 12552718.75000 148409.37500
                3001 21111536.58537 181889.02439
                4001 21512090.90909 208479.69697
                5001 23947714.28571 218600.00000
      영구임대      1001 6264200.00000  101022.00000
      행복주택      2001 30560000.00000 145160.00000
대구광역시 공공임대(10년) 4001 29500000.00000 387500.00000
                5001 37923076.92308 460769.23077
                6001 48428571.42857 545714.28571
                7001 58500000.00000 575000.00000
      국민임대      1001 9373500.00000  149831.66667
                2001 15261315.78947 141126.84211
                3001 23425541.66667 187127.91667
                4001 30020250.00000 245805.00000
                5001 33416000.00000 247303.33333
      영구임대      1001 12604500.00000 107715.00000
      행복주택      1001 18891428.57143 99528.57143 
                2001 30060000.00000 157365.00000
대전광역시 공공임대(10년) 4001 28013000.00000 408600.00000
                5001 37429500.00000 517740.00000
      공공임대(50년) 2001 13701000.00000 188240.00000
                4001 20688000.00000 225100.00000
      공공임대(5년)  2001 23000000.00000 340000.00000
                3001 26000000.00000 380000.00000
                5001 32000000.00000 480000.00000
      공공임대(분납)  4001 54612000.00000 131130.00000
                5001 63543000.00000 152580.00000
      국민임대      1001 14494250.00000 135590.00000
                2001 20358205.88235 156428.82353
                3001 33534150.00000 200705.00000
                4001 30847535.71429 208043.21429
                5001 29593600.00000 204033.00000
      영구임대      1001 6815565.21739  82638.69565 
                2001 17814200.00000 102456.00000
                4001 11574000.00000 159960.00000
      행복주택      1001 21813333.33333 105000.00000
                2001 33640000.00000 162000.00000
부산광역시 공공임대(10년) 6001 57000000.00000 550000.00000
                7001 70000000.00000 620000.00000
      공공임대(50년) 2001 13253500.00000 168575.00000
      국민임대      2001 17866736.84211 165991.05263
                3001 27492625.00000 235268.75000
                4001 38557333.33333 243344.44444
                5001 28780125.00000 249321.25000
      영구임대      1001 6904708.33333  95502.50000 
                2001 8848000.00000  122290.00000
      행복주택      1001 23000000.00000 111165.00000
                2001 38400000.00000 185600.00000

In [126]:
train_df_apt_null[["지역","공급유형","평수범주","임대보증금","임대료"]].groupby(["지역","공급유형","평수범주"]).max()
train_df_apt_null[["지역","공급유형","전용면적","평수범주","임대보증금","임대료"]]
# 범위에 없는 결측치도 발생..
# Test + Train 합치고
# 에러 데이터 삭제후 진행 


,지역,공급유형,전용면적,평수범주,임대보증금,임대료
2331,대전광역시,공공분양,74.94000,6001,NaN,NaN
2332,대전광역시,공공분양,74.94000,6001,NaN,NaN
2333,대전광역시,공공분양,74.94000,6001,NaN,NaN
2334,대전광역시,공공분양,84.94000,7001,NaN,NaN
2335,대전광역시,공공분양,84.94000,7001,NaN,NaN
2336,대전광역시,공공분양,84.96000,7001,NaN,NaN
2337,대전광역시,공공분양,84.97000,7001,NaN,NaN
2547,부산광역시,국민임대,24.72000,1001,NaN,NaN
2548,부산광역시,국민임대,24.79000,1001,NaN,NaN
2549,부산광역시,국민임대,26.83000,1001,NaN,NaN


In [ ]:

# 임대료가 Null인경우 
print("임대료가 Null 데이터 : \n {}".format(train_df[train_df["임대료"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
# 상가 : 562건 , 아파트 : 62건
# 임대료는 분양의경우 존재 하지 않기 때문에 공급유형이 분양인 경우 0으로 값을 보정할 수 있다.

In [ ]:
# 3. 지하철역수 가 nan인 데이터에서 지역 정보에 따른 데이터확인 여하여 Null 보정
print("---------------------------------------------------------")
print("지하철역 Null 데이터 : \n {}".format(train_df[train_df["도보 10분거리 내 지하철역 수(환승노선 수 반영)"].isnull() == True].groupby(["임대건물구분"]).count()["단지코드"]))
# 경남 : 4건 , 대전 : 93건 , 충청남도: 114건 
# 부산은 따로 표현되므로 지하철의 경우 경남, 충청남도는 지하철 노선이 존재하지 않으므로 0값 으로 채울 수 있음
# 경남, 충남은 Null값 0을 보정 가능
# 대전 Case 세부분할 확인

In [237]:
# 4. 버스가 Nan 인 데이터 지역정보 확인
print("---------------------------------------------------------")
print("버스정류장 Null 데이터 : \n {}".format(train_df[train_df["도보 10분거리 내 버스정류장 수"].isnull() == True].groupby(["지역"]).count()["단지코드"]))
# 경상남도 : 4건 



# 버스정류장의 경우 신축아파트의 경우 노선이 확정되지 않아 없는 경우가 존재할 수 있으나, 
# 데이터 확인결과 공공임대 아파트인 점을 보았을때, 표기오류로 판단할 수 있다. 
# 0으로 값을 채워넣기 보단 공공임대 아파트의 평균 정류장수를 사용하여 결측치를 보정할 수 있을 것같다.
print(train_df[train_df["도보 10분거리 내 버스정류장 수"].isnull() == True]) 

# 총 251건의 공공임대 아파트 유형 존재
print(len(train_df[train_df["공급유형"].str.contains("공공임대")]))

print("공공임대 아파트의 평균 버스정류장 수 : {} 개".format(math.floor(train_df[train_df["공급유형"].str.contains("공공임대")]["도보 10분거리 내 버스정류장 수"].dropna().mean())))
# 평균 3개의 버스정류장이 존재 
# 버스정류장의 Null 값은 3으로 보정하여 수행

---------------------------------------------------------
버스정류장 Null 데이터 : 
 지역
경상남도    4
Name: 단지코드, dtype: int64
       단지코드  총세대수 임대건물구분    지역       공급유형   전용면적  전용면적별세대수   공가수 자격유형     임대보증금     임대료  도보 10분거리 내 지하철역 수(환승노선 수 반영)  도보 10분거리 내 버스정류장 수  단지내주차면수  등록차량수
2315  C1649  575   아파트    경상남도  공공임대(10년)  74.97  80        15.0  A    46000000  456000 NaN                           NaN                  1066.0   855.0
2316  C1649  575   아파트    경상남도  공공임대(10년)  84.95  124       15.0  A    57000000  462000 NaN                           NaN                  1066.0   855.0
2317  C1649  575   아파트    경상남도  공공임대(10년)  84.96  289       15.0  A    57000000  462000 NaN                           NaN                  1066.0   855.0
2318  C1649  575   아파트    경상남도  공공임대(10년)  84.98  82        15.0  A    57000000  462000 NaN                           NaN                  1066.0   855.0
251
공공임대 아파트의 평균 버스정류장 수 : 3 개


In [180]:
# 임대료 보정하기 전에 Null 값의 유형을 좀 파악 해보자!

train_df["임대보증금"] = train_df["임대보증금"].fillna("0")
train_df["임대료"] = train_df["임대료"].fillna("0")

# 지하철역수 및 버정류장수 Null 보정

train_df["도보 10분거리 내 버스정류장 수"] = train_df["도보 10분거리 내 버스정류장 수"].fillna(0)
train_df["도보 10분거리 내 버스정류장 수"] = train_df["도보 10분거리 내 버스정류장 수"].fillna(0)

In [181]:
# 임대 보증금과 임대료 Object type -> 수치형으로 변환
train_df = train_df.astype({"임대보증금" : int, "임대료" : int}) 

train_df.describe().T


,count,mean,std,min,25%,50%,75%,max
총세대수,2952.0,8.866612e+02,5.135402e+02,26.00,513.50,779.00,1.106000e+03,2568.0
전용면적,2952.0,4.475722e+01,3.187428e+01,12.62,32.10,39.93,5.156250e+01,583.4
전용면적별세대수,2952.0,1.027480e+02,1.326402e+02,1.00,14.00,60.00,1.440000e+02,1865.0
공가수,2952.0,1.292107e+01,1.077883e+01,0.00,4.00,11.00,2.000000e+01,55.0
임대보증금,2952.0,2.157700e+07,2.212032e+07,0.00,8731000.00,17477000.00,2.820275e+07,216423000.0
임대료,2952.0,1.566010e+05,1.404528e+05,0.00,75520.00,135960.00,2.100350e+05,1058030.0
도보 10분거리 내 지하철역 수(환승노선 수 반영),2741.0,1.765779e-01,4.274079e-01,0.00,0.00,0.00,0.000000e+00,3.0
도보 10분거리 내 버스정류장 수,2952.0,3.690718e+00,2.646368e+00,0.00,2.00,3.00,4.000000e+00,20.0
단지내주차면수,2952.0,6.016684e+02,3.964071e+02,13.00,279.25,517.00,8.230000e+02,1798.0
등록차량수,2952.0,5.597683e+02,4.333750e+02,13.00,220.00,487.00,7.700000e+02,2550.0
